In [1]:
import pandas as pd

pipeline_export = pd.read_csv('/Users/smarshall/Desktop/threat_pipeline_export.csv')

In [2]:
pipeline_export['json_status'] = pipeline_export['status'].str[1:-1]

dropped_nas = pipeline_export[pipeline_export['status'].notna()]

In [8]:
import json

def convert_json(my_table):
    shell_df = pd.DataFrame()
    for i in range(len(my_table)):
        extracted_value = my_table.iloc[i]['json_status']
        json_value = json.loads(extracted_value)
        normalized_value = pd.json_normalize(json_value)
        shell_df = pd.concat([shell_df, normalized_value])
    # sub_shell = shell_df[['binary_result.pipeline_task_id', 'binary_result.task_id', 'regular_result.task_id', 'binary_result.text_data','binary_result.status', 'regular_result.status']]
    # sub_shell.columns = ['pipeline_task_id', 'binary_task_id', 'multi_task_id', 'text_data', 'binary_result', 'multi_result']
    return shell_df


In [16]:
def convert_json_dfs(my_table):
    def normalize_json(row):
        json_value = pd.json_normalize(row['json_status'])
        return json_value
        
    dfs = my_table.apply(normalize_json, axis=1)
    shell_df = pd.concat(dfs, ignore_index=True)
    return shell_df

In [9]:
try1 = convert_json(dropped_nas)

In [ ]:
new_fuunc = convert_json_dfs(dropped_nas)

In [ ]:
# merge 
merged_with_callback = pd.merge(try1, pipeline_export[['callback_metadata', 'text_data']], how = 'left', on='text_data')

merged_with_callback['duped'] = merged_with_callback.duplicated(subset=['pipeline_task_id'])

merged_with_callback.loc[merged_with_callback['duped'] == True]